<a href="https://colab.research.google.com/github/Natural-State/ndvi-test/blob/main/NDVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract NDVI for an AOI and export to Google Drive

See [here](https://developers.google.com/earth-engine/guides/python_install#syntax) for differences between Javascript and Python syntax

Main [tutorial](https://courses.spatialthoughts.com/end-to-end-gee.html#module-6-google-earth-engine-python-api) here

## Import gee and authenticate

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=32DZV1vhoEIZCLlyqA2054-rxrQgrtO5uwaSRhj2Uuo&tc=ZQ-RgbtuyO9fPa741c7acynpSi1f3O3sLG6gvAcXQ3s&cc=81tIqK_xDOn1O9gTO_N4qpPNsM7C8t56O9QK-NAHrCk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh4DSbdELXu15QPMeG_d_CBowpJmTZVikYOKk-HLg76XRFfbP-DQ_Qo

Successfully saved authorization token.


## Test API

In [ ]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


## Import EE assets

See `mod3_7_import_export_data.ipynb` for local import of shapefiles (convert to EE object)

Google Drive can be mounted after which files are accessible at `content/drive/MyDrive`

In [ ]:
aoi = ee.FeatureCollection("users/dominic-henry/Mount_Kenya_Rangelands/Ngare_Ndare")

## Import image collection

In [ ]:
sentinel2 = ee.ImageCollection("COPERNICUS/S2")

## NDVI processing

In [ ]:
# Date parameters
start_date = ee.Date('2021-01-01')
end_date = ee.Date('2021-12-31')

# Layer naming
aoi_name = aoi.getString("system:id")
aoi_name = aoi_name.slice(aoi_name.rindex("/"), aoi_name.length()).replace('/','')
aoi_name = aoi_name.cat('_NDVI_')
aoi_name = aoi_name.cat(start_date.format('YYYY'))
print(aoi_name.getInfo())

Ngare_Ndare_NDVI_2021


In [ ]:
ndvi_2020 = sentinel2.filterDate(start_date, end_date)

def addNDVI(image):
  imgb = image.select('B.*')
  ndvi = imgb.normalizedDifference(['B8','B4']).rename('NDVI')
  return image.addBands(ndvi)

ndvi_2020 = ndvi_2020.map(addNDVI).mean().clip(aoi)

In [ ]:
# Select only NDVI band
ndvi_2020 = ndvi_2020.select('NDVI')

## Export data - create task

`filenamePrefix` should be in format: place_layer_timeperiod

In [ ]:
task = ee.batch.Export.image.toDrive(image = ndvi_2020,
                                     region = aoi.geometry().bounds(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = aoi_name.getInfo() + "_test",
                                     scale = 30,
                                     maxPixels = 10e12
                                     )

# print(aoi_name.getInfo() + "test")

## Start task

In [ ]:
task.start()


## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
task.status()

{'state': 'COMPLETED',
 'description': 'EXPORT IMAGE TO DRIVE',
 'creation_timestamp_ms': 1678116207430,
 'update_timestamp_ms': 1678116234179,
 'start_timestamp_ms': 1678116220802,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/1KNdzJm62w6HMbH2nNTYpHwFgX9TuEDxk'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 17.507537841796875,
 'id': 'HEOXJVXLTZBQDSTRCTLKUSNZ',
 'name': 'projects/earthengine-legacy/operations/HEOXJVXLTZBQDSTRCTLKUSNZ'}

## Mapping images

In [ ]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

 # Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': -1,
  'max': 1,
  'palette': ['red', 'yellow', 'green']}

# Create a folium map object.
my_map = folium.Map(location=[0.1591, 37.3729], zoom_start=12)

# Add the NDVI image to the map object.
my_map.add_ee_layer(ndvi_2020, vis_params, 'NDVI')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)